In [15]:
import base64
import os.path as op
from io import BytesIO

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np

In [16]:
def load_and_reorient(filename):
    img = nib.load(filename)
    data, aff = img.get_data(), img.affine
    data = reorient_array(data, aff)
    return data

def reorient_array(data, aff):
    # rearrange the matrix to RAS orientation
    orientation = nib.orientations.io_orientation(aff)
    data_RAS = nib.orientations.apply_orientation(data, orientation)
    # In RAS
    return nib.orientations.apply_orientation(
        data_RAS,
        nib.orientations.axcodes2ornt("IPL")
    )

def get_middle_slices(data, slice_direction):
    slicer = {"ax": 0, "cor": 1, "sag": 2}
    all_data_slicer = [slice(None), slice(None), slice(None)]
    num_slices = data.shape[slicer[slice_direction]]
    slice_num = int(num_slices / 2)
    all_data_slicer[slicer[slice_direction]] = slice_num
    tile = data[tuple(all_data_slicer)]

    # make it a square
    N = max(tile.shape[:2])
    tile = reshape3D(tile, N)

    return tile

def reshape3D(data, n=256):
    return np.pad(data, (
        (
            (n-data.shape[0]) // 2,
            ((n-data.shape[0]) + (data.shape[0] % 2 > 0)) // 2
        ),
        (
            (n-data.shape[1]) // 2,
            ((n-data.shape[1]) + (data.shape[1] % 2 > 0)) // 2
        ),
        (0, 0)
    ), "constant", constant_values=(0, 0))


def create_sprite_from_tiles(tile, out_file=None, as_bytes=False):
    num_slices = tile.shape[-1]
    N = nearest_square(num_slices)
    M = int(np.ceil(num_slices/N))
    # tile is square, so just make a big arr
    pix = tile.shape[0]

    if len(tile.shape) == 3:
        mosaic = np.zeros((N*tile.shape[0], M*tile.shape[0]))
    else:
        mosaic = np.zeros((N*tile.shape[0], M*tile.shape[0], tile.shape[-2]))

    mosaic[:] = np.nan
    helper = np.arange(N*M).reshape((N, M))

    for t in range(num_slices):
        x, y = np.nonzero(helper == t)
        xmin = x[0] * pix
        xmax = (x[0] + 1) * pix
        ymin = y[0] * pix
        ymax = (y[0] + 1) * pix

        if len(tile.shape) == 3:
            mosaic[xmin:xmax, ymin:ymax] = tile[:, :, t]
        else:
            mosaic[xmin:xmax, ymin:ymax, :] = tile[:, :, :, t]

    if as_bytes:
        img = mplfig(mosaic, out_file, as_bytes=as_bytes)
        return dict(img=img, N=N, M=M, pix=pix, num_slices=num_slices)

    if out_file:
        img = mplfig(mosaic, out_file), N, M, pix, num_slices

    return dict(mosaic=mosaic, N=N, M=M, pix=pix, num_slices=num_slices)

def nearest_square(limit):
    answer = 0
    while (answer+1)**2 < limit:
        answer += 1
    if (answer ** 2) == limit:
        return answer
    else:
        return answer + 1

def mplfig(data, outfile=None, as_bytes=False):
    fig = plt.figure(frameon=False, dpi=data.shape[0])
    fig.set_size_inches(float(data.shape[1])/data.shape[0], 1)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(data, aspect=1, cmap=plt.cm.Greys_r)  # previous aspect="normal"
    if outfile:
        fig.savefig(outfile, dpi=data.shape[0], transparent=True)
        plt.close()
        return outfile
    if as_bytes:
        IObytes = BytesIO()
        plt.savefig(IObytes, format='png', dpi=data.shape[0], transparent=True)
        IObytes.seek(0)
        base64_jpgData = base64.b64encode(IObytes.read())
        return base64_jpgData.decode("ascii")

In [17]:
dwi_file = "../data/sub-CMH0171/ses-01/dwi/sub-CMH0171_ses-01_acq-singleshell60dir_dwi.nii.gz"

In [18]:
output = []

dwi = load_and_reorient(dwi_file)[:, :, :, 1:]

for orient in ['sag', 'ax', 'cor']:
    tile = get_middle_slices(dwi, orient)
    
    results = create_sprite_from_tiles(tile, as_bytes=True)
    results['img_type'] = '4dsprite'
    results['orientation'] = orient
    output.append(results)

In [32]:
output[0]

{'img': '...',
 'N': 8,
 'M': 8,
 'pix': 128,
 'num_slices': 64,
 'img_type': '4dsprite',
 'orientation': 'sag'}

In [1]:
from docopt import docopt
import pandas as pd
import os
import tempfile
import shutil
from glob import glob
import sys
import subprocess as proc

import datman as dm
import datman.utils

ModuleNotFoundError: No module named 'datman'

In [10]:
output_dir = '/KIMEL/tigrlab/scratch/smansour/COMPULS/dmriprep_output/dmripreproc'
dtifit_dir = '/KIMEL/tigrlab/scratch/mjoseph/tmp/compuls_tmp'
QCdir = os.path.join(dtifit_dir, 'QC')
tmpdirbase = os.path.join(QCdir,'tmp')
QC_bet_dir = os.path.join(QCdir,'BET')
QC_V1_dir = os.path.join(QCdir, 'directions')
QC_FM_dir = os.path.join(QCdir, 'FM')
QC_Mag_dir = os.path.join(QCdir, 'Mag')
QC_res_dir = os.path.join(QCdir, 'res')
QC_SH_dir = os.path.join(QCdir, 'SH')

dir_list = [tmpdirbase, QC_bet_dir, QC_V1_dir, QC_res_dir, QC_SH_dir]

In [3]:
for qc_dir in dir_list:
    if not os.path.exists(qc_dir):
        os.makedirs(qc_dir)

PermissionError: [Errno 13] Permission denied: '/scratch'

In [4]:
grad_out = os.path.join(tmpdirbase,'ramp.gif')
create_gradient_file(grad_out,'red-yellow')

NameError: name 'create_gradient_file' is not defined

In [20]:
len(SHpics)

157

In [7]:
maskpics = []
V1pics = []
Respics = []
SHpics = []

In [14]:
maskpics = sorted(glob('/KIMEL/tigrlab/scratch/mjoseph/tmp/compuls_tmp/QC/BET/*'))
V1pics = sorted(glob('/KIMEL/tigrlab/scratch/mjoseph/tmp/compuls_tmp/QC/directions/*'))
Respics = sorted(glob('/KIMEL/tigrlab/scratch/mjoseph/tmp/compuls_tmp/QC/res/*'))
SHpics = sorted(glob('/KIMEL/tigrlab/scratch/mjoseph/tmp/compuls_tmp/QC/SH/*'))

In [6]:
allFAmaps = sorted(glob('{}/sub-*/ses-*/dwi/dtifit__FA*'.format(output_dir)))

In [20]:
for FAmap in allFAmaps:
    subid = FAmap.split('/')[-4]
    sesid = FAmap.split('/')[-3]
    tmpdir = os.path.join(tmpdirbase, subid, sesid)
    if not os.path.exists(tmpdir):
        os.makedirs(tmpdir)
    basename = '{}_{}_'.format(subid, sesid)
    pathbase = FAmap.replace('dtifit__FA.nii.gz','')
    pathdir = os.path.dirname(pathbase)

    maskpic = os.path.join(QC_bet_dir,basename + 'b0_bet_mask.gif')
    maskpics.append(maskpic)
    
    if not os.path.exists(maskpic):
        mask_overlay(os.path.join(pathdir,'{}_dwi_denoised_resized_avg_b0_brain.nii.gz'.format(subid)), 
                     os.path.join(pathdir,'{}_dwi_denoised_resized_avg_b0_brain_mask.nii.gz'.format(subid)), maskpic)

    V1pic = os.path.join(QC_V1_dir,basename + 'dtifit_V1.gif')
    V1pics.append(V1pic)
    
    if not os.path.exists(V1pic):
        V1_overlay(FAmap,pathbase + 'dtifit__V1.nii.gz', V1pic)

In [21]:
qchtml = open(os.path.join(QCdir,'qc_BET.html'),'w')
qchtml.write('<HTML><TITLE>DTIFIT BET QC page</TITLE>')
qchtml.write('<BODY BGCOLOR=#333333>\n')
qchtml.write('<h1><font color="white">DTIFIT BET QC page</font></h1>')
for pic in maskpics:
    relpath = os.path.relpath(pic,QCdir)
    qchtml.write('<a href="'+ relpath + '" style="color: #99CCFF" >')
    qchtml.write('<img src="' + relpath + '" "WIDTH=800" > ')
    qchtml.write(relpath + '</a><br>\n')
qchtml.write('</BODY></HTML>\n')
qchtml.close() # you can omit in most cases as the destructor will call it

## write an html page that shows all the V1 pics
qchtml = open(os.path.join(QCdir,'qc_directions.html'),'w')
qchtml.write('<HTML><TITLE>DTIFIT directions QC page</TITLE>')
qchtml.write('<BODY BGCOLOR=#333333>\n')
qchtml.write('<h1><font color="white">DTIFIT directions QC page</font></h1>')
for pic in V1pics:
    relpath = os.path.relpath(pic,QCdir)
    qchtml.write('<a href="'+ relpath + '" style="color: #99CCFF" >')
    qchtml.write('<img src="' + relpath + '" "WIDTH=800" > ')
    qchtml.write(relpath + '</a><br>\n')
qchtml.write('</BODY></HTML>\n')
qchtml.close() # you can omit in most cases as the destructor will call it

In [12]:
for FAmap in allFAmaps:
    subid = FAmap.split('/')[-4]
    sesid = FAmap.split('/')[-3]
    tmpdir = os.path.join(tmpdirbase, subid, sesid)
    if not os.path.exists(tmpdir):
        os.makedirs(tmpdir)
    basename = '{}_{}_'.format(subid, sesid)
    pathbase = FAmap.replace('dtifit__FA.nii.gz','')
    pathdir = os.path.dirname(pathbase)

    SHtmp = os.path.join(tmpdir,'SH.gif')
    SHpic = os.path.join(QC_SH_dir,basename + 'SH.gif')
    SHpics.append(SHpic)
    SH_overlay(os.path.join(pathdir,'eddy_corrected_noise_masked.nii.gz'), SHtmp, grad_out)
    if not os.path.exists(SHpic):
        gif_gridtoline(SHtmp,SHpic)
        
    SEtmp = os.path.join(tmpdir,'SE.gif')
    Respic = os.path.join(QC_res_dir,basename + 'sse.gif')
    Respics.append(Respic)
    SSE_overlay(os.path.join(pathdir,'dtifit__sse.nii.gz'), SEtmp, grad_out)
    if not os.path.exists(Respic):
        gif_gridtoline(SEtmp,Respic)

In [22]:
# write an html page that shows all the SH residual pics
qchtml = open(os.path.join(QCdir,'qc_SH.html'),'w')
qchtml.write('<HTML><TITLE>SH residual QC page</TITLE>')
qchtml.write('<BODY BGCOLOR=#333333>\n')
qchtml.write('<h1><font color="white">SH residual QC page</font></h1>')
for pic in SHpics:
    relpath = os.path.relpath(pic,QCdir)
    qchtml.write('<a href="'+ relpath + '" style="color: #99CCFF" >')
    qchtml.write('<img src="' + relpath + '" "WIDTH=800" > ')
    qchtml.write(relpath + '</a><br>\n')
qchtml.write('</BODY></HTML>\n')
qchtml.close() # you can omit in most cases as the destructor will call it

## write an html page that shows all the Res pics
qchtml = open(os.path.join(QCdir,'qc_res.html'),'w')
qchtml.write('<HTML><TITLE>DTIFIT residual QC page</TITLE>')
qchtml.write('<BODY BGCOLOR=#333333>\n')
qchtml.write('<h1><font color="white">DTIFIT residual QC page</font></h1>')
for pic in Respics:
    relpath = os.path.relpath(pic,QCdir)
    qchtml.write('<a href="'+ relpath + '" style="color: #99CCFF" >')
    qchtml.write('<img src="' + relpath + '" "WIDTH=800" > ')
    qchtml.write(relpath + '</a><br>\n')
qchtml.write('</BODY></HTML>\n')
qchtml.close() # you can omit in most cases as the destructor will call it

In [5]:
def gif_gridtoline(input_gif,output_gif):
    '''
    uses imagemagick to take a grid from fsl slices and convert to one line (like in slicesdir)
    '''
    dm.utils.run(['convert',input_gif, '-resize', '384x384',input_gif])
    dm.utils.run(['convert', input_gif,\
        '-crop', '100x33%+0+0', os.path.join(tmpdir,'sag.gif')])
    dm.utils.run(['convert', input_gif,\
        '-crop', '100x33%+0+128', os.path.join(tmpdir,'cor.gif')])
    dm.utils.run(['convert', input_gif,\
        '-crop', '100x33%+0+256', os.path.join(tmpdir,'ax.gif')])
    dm.utils.run(['montage', '-mode', 'concatenate', '-tile', '3x1', \
        os.path.join(tmpdir,'sag.gif'),\
        os.path.join(tmpdir,'cor.gif'),\
        os.path.join(tmpdir,'ax.gif'),\
        os.path.join(output_gif)])

In [6]:
def mask_overlay(background_nii,mask_nii, overlay_gif):
    '''
    use slices from fsl to overlay the mask on the background (both nii)
    then make the grid to a line for easier scrolling during QC
    '''
    dm.utils.run(['slices', background_nii, mask_nii, '-o', os.path.join(tmpdir,'B0masked.gif')])
    gif_gridtoline(os.path.join(tmpdir,'B0masked.gif'),overlay_gif)

In [7]:
def V1_overlay(background_nii,V1_nii, overlay_gif):
    '''
    use fslsplit to split the V1 image and take pictures of each direction
    use slices from fsl to get the background and V1 picks (both nii)
    recolor the V1 image using imagemagick
    then make the grid to a line for easier scrolling during QC
    '''
    dm.utils.run(['slices',background_nii,'-o',os.path.join(tmpdir,"background.gif")])
    dm.utils.run(['fslmaths',background_nii,'-thr','0.15','-bin',os.path.join(tmpdir,'FAmask.nii.gz')])
    dm.utils.run(['fslsplit', V1_nii, os.path.join(tmpdir,"V1")])
    for axis in ['0000','0001','0002']:
        dm.utils.run(['fslmaths',os.path.join(tmpdir,'V1'+axis+'.nii.gz'), '-abs', \
            '-mul', os.path.join(tmpdir,'FAmask.nii.gz'), os.path.join(tmpdir,'V1'+axis+'abs.nii.gz')])
        dm.utils.run(['slices',os.path.join(tmpdir,'V1'+axis+'abs.nii.gz'),'-o',os.path.join(tmpdir,'V1'+axis+'abs.gif')])
        # docmd(['convert', os.path.join(tmpdir,'V1'+axis+'abs.gif'),\
        #         '-fuzz', '15%', '-transparent', 'black', os.path.join(tmpdir,'V1'+axis+'set.gif')])
    dm.utils.run(['convert', os.path.join(tmpdir,'V10000abs.gif'),\
        os.path.join(tmpdir,'V10001abs.gif'), os.path.join(tmpdir,'V10002abs.gif'),\
        '-set', 'colorspace', 'RGB', '-combine', '-set', 'colorspace', 'sRGB',\
        os.path.join(tmpdir,'dirmap.gif')])
    gif_gridtoline(os.path.join(tmpdir,'dirmap.gif'),overlay_gif)

In [8]:
def SSE_overlay(sse,out,grad):
    '''
    Arguments:
        sse                        Full path to SSE file
        out                        Full path to output
        grad                       Full path to gradient look-up map

    Steps:
    1. Clever/Hacky thresholding so maximum intensity is 2
    2. Generate slices
    3. Use gradient map to color greyscale image
    4. Background filling with 0 fuzziness to prevent leakage
    '''
    slice_out = out.replace('.nii.gz','.gif')
    cmd1 = 'fslmaths {} -sub 3 -mul -1 -thr 0 -mul -1 -add 3 {}'.format(sse,out)
    cmd2 = 'slices {} -o {}'.format(out, slice_out)
    cmd3 = 'convert {} {} -clut {}'.format(slice_out, grad, slice_out)
    cmd4 = 'convert {} -fill black -draw "color 0,0 floodfill" {}'.format(slice_out,slice_out)
    cmdlist = [cmd1, cmd2, cmd3, cmd4]
    outputs = [call(c) for c in cmdlist]
    return

In [23]:
def SH_overlay(SH,out,grad):
    '''
    Arguments:
        SH                         Full path to SSE file
        out                        Full path to output
        grad                       Full path to gradient look-up map

    Steps:
    1. Clever/Hacky thresholding so maximum intensity is 2
    2. Generate slices
    3. Use gradient map to color greyscale image
    4. Background filling with 0 fuzziness to prevent leakage
    '''
    slice_out = out.replace('.nii.gz','.gif')
    cmd1 = 'fslmaths {} -nan {}'.format(SH, out)
    cmd2 = 'fslmaths {} -sub 0.3 -mul -1 -thr 0 -mul -1 -add 0.3 {}'.format(out,out)
    cmd3 = 'slices {} -o {}'.format(out, slice_out)
    cmd4 = 'convert {} {} -clut {}'.format(slice_out, grad, slice_out)
    cmd5 = 'convert {} -fill black -draw "color 0,0 floodfill" {}'.format(slice_out,slice_out)
    cmdlist = [cmd1, cmd2, cmd3, cmd4, cmd5]
    outputs = [call(c) for c in cmdlist]
    return

In [10]:
def create_gradient_file(output,color):
    '''
    Arguments:
        output                    Full path to output file
        color                     String argument of Image-Magick 'color:color'
    '''

    cmd = 'convert -size 10x20 gradient:{} {}'.format(color,output)
    call(cmd)
    return

In [11]:
def call(cmd):
    p = proc.Popen(cmd,shell=True,stdin=proc.PIPE, stderr=proc.PIPE)
    std, err = p.communicate()

    if p.returncode:
        print('{} failed with error {}'.format(cmd,err))
    return

In [12]:
def get_sse(sub):
    sse = '{}_dtifit_sse.nii.gz'.format(sub)
    return os.path.join(dtifitdir,sub,sse)